Define values for Data Points

In [22]:
!pip install numpy

In [28]:
def value_func(i):
    return -2+0.2*i

def value_gen(max_val):
#    i_list=[x for x in range(max_val+1)]  # values that i and j can take
#   print(i_list)
    i_list = np.linspace(0, 20, num = 21,dtype=int)
    print("Range of i and j is : ",i_list,"Type returned", type(i_list))
    values_list=list(map(value_func,i_list))
    return values_list

values_list=value_gen(20)

Range of i and j is :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20] Type returned <class 'numpy.ndarray'>


In [30]:
def random_train_samples(value_list):
    train_samples=np.random.choice(value_list,size=441,replace=True)
    print(train_samples)
    print("Shape is:",train_samples.shape)
    return random_train_samples
random_train_samples(values_list)

[ 1.2  0.  -0.8 -0.8  1.2 -1.2 -2.  -1.6  1.2  0.6 -0.4  1.6 -1.8 -1.
  1.8 -0.2 -1.4  0.4  2.   1.8  0.4  1.2  0.  -1.6 -0.2 -2.   1.  -1.6
 -0.6  1.  -0.2 -1.2 -2.  -0.4  1.8  0.2  1.8 -2.   0.8 -1.   2.   1.8
  1.4 -1.   0.8 -0.2  1.  -0.8  1.8  0.6 -1.8  0.2  0.2  2.  -0.6  0.2
  0.6  2.   0.8  0.8 -0.2  0.  -0.4  0.2  1.6 -0.4  0.4  1.  -0.8 -1.8
 -0.6 -2.   1.8 -1.2  1.4 -1.6 -1.2 -0.8 -0.2  2.  -2.  -1.2 -0.4 -0.4
  1.4 -1.6 -1.4  1.  -1.   2.   0.8  0.6 -1.4  1.2 -0.8  2.   1.6 -0.8
 -0.2  1.6  1.4 -0.8 -1.4 -0.6 -1.2  1.6  0.2 -1.8 -1.  -1.8  0.   0.8
  0.6 -1.6 -1.  -0.2  1.2 -2.  -0.8 -0.8 -0.2 -1.8 -0.2  1.8 -0.4 -0.8
  1.8  0.6  1.   0.  -1.6  1.8 -0.4 -1.6  2.   0.   1.4 -1.  -0.8 -0.2
  0.2  0.4 -1.  -1.  -0.2 -1.  -1.2  0.4  1.6 -0.2  0.4 -1.2  2.  -1.6
 -1.8  1.2  1.   1.  -0.8  1.4 -1.   0.4  1.6 -1.8  1.8  1.2  0.2  2.
  0.4 -0.8 -0.2 -1.4  1.  -1.  -0.4 -1.8 -0.4 -1.6 -2.  -0.6 -0.6  2.
  1.8  0.2 -0.8 -1.6  1.4 -1.  -1.2 -1.4  0.2  1.8  1.2 -0.8  1.   1.6
 -0.2  1.

<function __main__.random_train_samples(value_list)>